Exploring Hacker News Posts

On Hacker News users submit Ask HN posts to ask the "Hacker News" community a specific question and "Show HN" posts to show a project, product, or just something interesting.

We'll compare these two types of posts to determine the following:

Do Ask HN or Show HN receive more comments on average?
Do posts created at a certain time receive more comments on average?

In [1]:
import csv

with open('hacker_news.csv', 'r') as file:
    reader = csv.reader(file)
    hn = list(reader)
headers = hn[0]
print(headers)

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


In [ ]:
del hn[0]
    
print(hn[:5])

[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


In [11]:
ask_posts = list()
show_posts = list()
other_posts = list()



for post in hn:
    title = post[1]
    if title.lower().startswith('ask hn'):
        ask_posts.append(post)
        """ print(title, 'ask') """
    elif title.lower().startswith('show hn'):
        show_posts.append(post)
        """ print(title, 'show') """
    else:
        other_posts.append(post)
        """ print(title, 'other') """

print("Number of Ask HN posts:", len(ask_posts))
print("Number of Show HN posts:", len(show_posts))
print("Number of other posts:", len(other_posts))

Number of Ask HN posts: 1744
Number of Show HN posts: 1162
Number of other posts: 17194


In [18]:
total_ask_comments = 0
total_show_comments = 0

for askcomm in ask_posts:
    comments: int = int(askcomm[4])
    total_ask_comments += comments

avg_ask_comments = total_ask_comments / len(ask_posts)

for showcomm in show_posts:
    comments: int = int(showcomm[4])
    total_show_comments += comments

avg_show_comments = total_show_comments / len(show_posts)

most_commented = ''

if avg_ask_comments != avg_show_comments:
    if avg_ask_comments > avg_show_comments:
        most_commented = 'Ask'
    elif avg_show_comments > avg_ask_comments:
        most_commented = 'Show'

comment = ''
if most_commented:
    comment = 'On average, ' + most_commented + ' posts have more comments.'
else:
    comment = 'On average, Show and Ask posts have the same number of comments.'

print(comment)


On average, Ask posts have more comments.
